# T1 and T2 fitting

In this Notebook, we will explore some basics of quantitative MRI, specifically T1 and T2 fitting.

**Note:** This Notebook contains some interactive elements. If you select a different voxel in the image during data load, re-run the subsequent cells to see the updated results.

## Initialization

The following code imports the required packages and defines a couple of utility functions.

In [1]:
import ipympl
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

def pickPoint(dataset, timebase):
    imaFig = plt.figure()
    plt.title('Click on the image to see the time course')
    plt.imshow(t1dataset[:,:,0])
    plotFig = plt.figure()
    plt.title('Time course')
    def onclick(event):
        global timeseries
        row = int(event.ydata)
        col = int(event.xdata)
        ts = np.squeeze(dataset[row,col,:])
        plt.figure(plotFig.number)
        plt.clf()
        plt.plot(timebase, ts)
        timeseries = ts

    cid = imaFig.canvas.mpl_connect('button_press_event', onclick)

def showFit(timebase, yValues, modelFun, params):
    plt.figure()
    plt.plot(timebase, yValues, 'bo')
    t = np.linspace(timebase[0], timebase[-1])
    fitted = np.array(list(map(lambda x: modelFun(x, *params), t)))
    plt.plot(t, fitted, 'k')
    
def fitError(pcov):
    return np.sqrt(np.diag(pcov))

## T1 fitting

### Data loading and visualization

Now we will load the T1 dataset. It is a real part dataset, so, in principle, it contains both negative and positive values. Negative values correspond to TIs that are shorter than the zero crossing for the considered tissue in an inversion recovery experiment: $TI \le T_1 \ln{2}$. To represent a typical scenario of a magnitude acquisition, we will take the absolute value of each voxel.

This dateset is an inversion-recovery turbo spin echo (real-part data), TR=15000ms.

In [2]:
t1dataset = np.load('t1dataset.npy')
ti = np.array([50., 100., 200., 500., 2000., 4000.])
t1dataset = np.abs(t1dataset)

We will now visualize the image. Click on a voxel to select it. The time course will appear below. See the behavior of different tissue types.

In [3]:
timeseries = t1dataset[130,70,:]
pickPoint(t1dataset, ti)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

### Curve fitting

We will now see how the time series is fit to a model curve to extract the T1 parameter.

The most basic T1 recovery function is the exponential one:

$S(TI) = M_0 (1-2\exp(-TI/T_1))$

Where $S$ is the signal as a function of inversion time, $M_0$ is the "proton density", actually containing everything that is not T1, i.e. coil sensitivity, etc

The following is the classical T1 model function written in python: you can play around with this. Maybe you can add an imperfect inversion term i.e. change the 2 to a free parameter.

In [4]:
def t1ModelFunction(ti, m0, t1):
    return m0*(1 - 2*np.exp(-ti/t1))

This is how the signal would look like if it followed the model function perfectly:

In [5]:
showFit(ti, t1ModelFunction(ti, 1000, 1000), t1ModelFunction, (1000,1000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

As you notice, the first few points are negative, and your time curve is all positive, because we assumed a magnitude acquisition.

In this case, we need to invert the first few points of the curve to be able to do a proper fit. We assume that all the points of the curve before the minimum should be negative (and this is always true because the recovery is monotonic, save for the effect of noise, which we will disregard now). The only question is the minimum point itself: should it be positive or negative?

To answer this question we will be fitting the curve twice, once with a positive minimum, and once with a negative minimum, and see what's better.

In [13]:
minPoint = np.argmin(timeseries)
timeSeries1 = np.copy(timeseries)

# first, fit the curve with the minimum point as positive
timeSeries1[0:minPoint] = -timeSeries1[0:minPoint]
fittedParams1, covariance = opt.curve_fit(t1ModelFunction, ti, timeSeries1, p0 = (100,1000))

m0 = fittedParams1[0]
t1 = fittedParams1[1]

# show the fit
showFit(ti, timeSeries1, t1ModelFunction, fittedParams1)
plt.title("T1: {:.1f}, T1 error: {:.1f}".format(t1, fitError(covariance)[1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0.5,1,'T1: 685.0, T1 error: 40.5')

In [14]:
# repeat the fit by assuming the minimum point as negative
timeSeries2 = np.copy(timeseries)
timeSeries2[0:minPoint+1] = -timeSeries2[0:minPoint+1]

fittedParams2, covariance = opt.curve_fit(t1ModelFunction, ti, timeSeries2, p0 = (100,1000))

# m0 is the "proton density", actually containing everything that is not T1,
# i.e. coil sensitivity, etc
m0 = fittedParams2[0]
t1 = fittedParams2[1]

# show the fit
showFit(ti, timeSeries2, t1ModelFunction, fittedParams2)
plt.title("T1: {:.1f}, T1 error: {:.1f}".format(t1, fitError(covariance)[1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0.5,1,'T1: 802.3, T1 error: 30.8')

Which one is the best fit?

## T2 Fitting

T2 fitting is relatively simple compared to T1 fitting, because we do not have to deal with negative values. Like before, let's load some data.

This dataset is a single-echo spin echo, TR=5000ms.

In [8]:
# This dataset is a single-echo spin echo, TR=5000ms
t2dataset = np.load('t2dataset.npy')
te = np.array([10., 25., 50., 90., 150., 250.])

As before, we will visualize the image. Click on a voxel to select it and see its time course.

In [9]:
timeseries = t2dataset[130,70,:]
pickPoint(t2dataset, te)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

### Curve fitting

As you can see above, the time course of a T2 decay is simpler than T1.

The most basic T2 decay function is also exponential:

$S(TE) = M_0 exp(-TE/T_2))$

Where $S$ is the signal as a function of the echo time, $M_0$ is, as before, the "proton density", actually containing everything that is not T1, i.e. coil sensitivity, etc

The following is the classical T2 model function written in python: you can play around with this. Maybe you can add a bias to take into account the noise at low signal levels.

In [10]:
def t2ModelFunction(te, m0, t2):
    return m0*np.exp(-te/t2)

Again this is a plot of a perfect T2 decay:

In [11]:
showFit(te, t2ModelFunction(te, 1000, 100), t2ModelFunction, (1000,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

We don't have to do any manipulation of the signal time course in this case, so a simple fit should be enough:

In [12]:
fittedParams, covariance = opt.curve_fit(t2ModelFunction, te, timeseries, p0 = (100,100))

m0 = fittedParams[0]
t2 = fittedParams[1]

# show the fit
showFit(te, timeseries, t2ModelFunction, fittedParams)
plt.title("T2: {:.1f}, T2 error: {:.1f}".format(t2, fitError(covariance)[1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0.5,1,'T2: 803.2, T2 error: 6.6')